In [10]:
eval_df.to_csv('/proj/NIRAL/users/siyangj/inference_eval_4.csv',index=None)

In [1]:
import tensorflow as tf
import sys
sys.path.append('/nas/longleaf/home/siyangj/NiftyNet/')
import niftynet as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import os
import re
import nibabel as nib
import sklearn as skl
import pandas as pd

pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 10000

INFO:tensorflow:TensorFlow version 1.9.0
CRITICAL:tensorflow:Optional Python module cv2 not found, please install cv2 and retry if the application fails.
INFO:tensorflow:Available Image Loaders:
['nibabel', 'skimage', 'pillow', 'simpleitk', 'dummy'].


In [2]:
def Dice(pred,true,labels=[0,1,2,3]):
    l = len(labels)
    ret = np.zeros(l)
    for i in range(l):
        lab = labels[i]
        p = pred==lab
        t = true==lab
        ret[i] = 2*np.logical_and(p,t).sum()/(p.sum()+t.sum())
    return ret

In [3]:
def inference_eval(model,inf_dir,data_path,eval_df,test_im,seg_suffix='seg/'):
    inf_re = re.compile(r'inference.*')
    im_re = re.compile(r'.*\.nii\.gz')
    model_path = '/proj/NIRAL/users/siyangj/model_{}/'.format(model)
    
    inf_ims = {f[0]:os.path.join(inf_dir,f) for f in os.listdir(inf_dir) 
            if (os.path.isfile(os.path.join(inf_dir, f)) and im_re.match(f) is not None)}
    l = len(eval_df.index)
    if len(eval_df.loc[eval_df['inf_dir']==inf_dir])>0:
        l = np.where(eval_df['inf_dir']==inf_dir)[0][0]
    eval_df.loc[l,'inf_dir']=inf_dir
    eval_df.loc[l,'model']=model
    eval_df.loc[l,'test_im']=test_im
    for im_num,im in inf_ims.items():
        true_im = os.path.join(data_path,seg_suffix,im_num+'.nrrd')
        true_arr = sitk.GetArrayFromImage(sitk.ReadImage(true_im))
        #print(true_arr.shape)
        inf_arr = sitk.GetArrayFromImage(sitk.ReadImage(im))
        #print(inf_arr.shape)
        c_mat = skl.metrics.confusion_matrix(true_arr.flatten(),inf_arr.flatten())
        p_mat = c_mat.T / c_mat.astype(np.float).sum(axis=1)
        eval_df.loc[l,im_num]=[p_mat.T]
        if im_num == test_im:
            eval_df.loc[l,'test_11'] = p_mat[1,1]
            eval_df.loc[l,'test_12'] = p_mat[2,1]
            eval_df.loc[l,'test_21'] = p_mat[1,2]
            eval_df.loc[l,'test_22'] = p_mat[2,2]
            eval_df.loc[l,'test_11+22'] = eval_df.loc[l,'test_11']+eval_df.loc[l,'test_22']
            eval_df.loc[l,'test_12+21'] = eval_df.loc[l,'test_12']+eval_df.loc[l,'test_21']
            Dice0123 = Dice(inf_arr,true_arr)
            eval_df.loc[l,'Dice0'] = Dice0123[0]
            eval_df.loc[l,'Dice1'] = Dice0123[1]
            eval_df.loc[l,'Dice2'] = Dice0123[2]
            eval_df.loc[l,'Dice3'] = Dice0123[3]

In [4]:
def model_eval(model,data_path,eval_df,test_im,seg_suffix='seg/'):
    model_path = '/proj/NIRAL/users/siyangj/model_{}/'.format(model)
    inf_re = re.compile(r'inference.*')
    im_re = re.compile(r'.*\.nii\.gz')
    inf_dirs = [f for f in os.listdir(model_path) 
            if (os.path.isdir(os.path.join(model_path, f)) 
                and inf_re.match(f) is not None)]
    for d in inf_dirs:
        inf_dir = os.path.join(model_path,d)
        inference_eval(model,inf_dir,data_path,eval_df,test_im,seg_suffix=seg_suffix)

In [5]:
eval_df = pd.DataFrame(columns=['model','inf_dir','test_im','test_11','test_12','test_21','test_22',
                                'test_11+22','test_12+21',
                                'Dice0','Dice1','Dice2','Dice3',
                                '1','2','3','4','5','6','7','8'],dtype=object)

for i in range (1,9):
    eval_df[str(i)] = eval_df[str(i)].astype(object)

In [5]:
eval_df = pd.read_csv('/proj/NIRAL/users/siyangj/inference_eval_4.csv')

In [6]:
data_path = '/proj/NIRAL/users/siyangj/myData/min_cut_outside/'
models = ['10112050',
         #'10112055',
         '10112056',
         '10112103']
for m in models:
    model_eval(m,data_path,eval_df,'1')

models = ['10112127','10112130']
data_path = '/proj/NIRAL/users/siyangj/myData/min_fill_outside/'

for m in models:
    model_eval(m,data_path,eval_df,'1')

In [7]:
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference_18900_10112055',
               '/proj/NIRAL/users/siyangj/myData/min_cut_outside/',eval_df,'1')
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055',
               '/proj/NIRAL/users/siyangj/myData/min_cut_outside/',eval_df,'1')
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference',
               '/proj/NIRAL/users/siyangj/myData/min_cut_outside/',eval_df,'1')
inference_eval('10112055','/proj/NIRAL/users/siyangj/model_10112055/inference_21900_normal',
               '/proj/NIRAL/users/siyangj/myData/min_normal/',eval_df,'1')

In [6]:
models = [#'10192000','10192002','10192004',
         '10202100','10202102','10202104','10202106',
         '10210050','10210052','10210054','10210056',
         '10210130','10210132','10210134','10210136']
data_path = '/proj/NIRAL/users/siyangj/myData/min_normal/'

for m in models:
    model_eval(m,data_path,eval_df,'1',seg_suffix='seg_inf_1/')

In [19]:
inference_eval('10210052','/proj/NIRAL/users/siyangj/model_10210052/inference_100000',
               '/proj/NIRAL/users/siyangj/myData/min_normal/',eval_df,'1')
inference_eval('10210052','/proj/NIRAL/users/siyangj/model_10210052/inference_110000',
               '/proj/NIRAL/users/siyangj/myData/min_normal/',eval_df,'1')

In [20]:
test_33 = [r['1'][0][3][3] for i,r in eval_df.iterrows()]

In [21]:
test_33

[0.7861897343557165,
 0.7174722317761547,
 0.8239828433851943,
 0.793766564832986,
 0.8083840815748478,
 0.8598134622777025,
 0.8622311691990208,
 0.8547049183644567,
 0.7924110304084812,
 0.75131001274607,
 0.8554231492908734,
 0.8064620551520424,
 0.9158882972112544,
 0.8747693572596297,
 0.903756250059752]

In [22]:
eval_df.insert(loc=9,column='test_33',value=test_33)

In [9]:
eval_df.sort_values('test_11+22',ascending=False).iloc[:,0:13]

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,Dice0,Dice1,Dice2,Dice3
14,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_normal,1,0.861763,0.124861,0.121004,0.822459,1.684223,0.245865,0.991524,0.843291,0.846411,0.833207
12,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055,1,0.861016,0.125160,0.120896,0.821308,1.682324,0.246056,0.995358,0.843021,0.846834,0.845988
35,10210050,/proj/NIRAL/users/siyangj/model_10210050/inference_30000,1,0.850768,0.138990,0.139546,0.831239,1.682007,0.278536,0.992319,0.825083,0.831247,0.809081
61,10210130,/proj/NIRAL/users/siyangj/model_10210130/inference_55000,1,0.788889,0.194381,0.081139,0.883951,1.672840,0.275520,0.994809,0.827094,0.851354,0.843518
72,10210132,/proj/NIRAL/users/siyangj/model_10210132/inference_70000,1,0.861237,0.128304,0.149105,0.802642,1.663880,0.277410,0.995638,0.825284,0.824976,0.832259
58,10210054,/proj/NIRAL/users/siyangj/model_10210054/inference_50000,1,0.829719,0.155244,0.121035,0.832751,1.662470,0.276280,0.995374,0.823699,0.834863,0.831108
55,10210050,/proj/NIRAL/users/siyangj/model_10210050/inference_60000,1,0.854372,0.132274,0.148030,0.797729,1.652101,0.280304,0.995481,0.820110,0.825959,0.833727
56,10210052,/proj/NIRAL/users/siyangj/model_10210052/inference_55400,1,0.862876,0.127103,0.177458,0.788410,1.651286,0.304561,0.995355,0.807689,0.810778,0.819124
54,10210050,/proj/NIRAL/users/siyangj/model_10210050/inference_47700,1,0.787571,0.184371,0.088806,0.862547,1.650118,0.273177,0.992172,0.821132,0.846978,0.825889
70,10210132,/proj/NIRAL/users/siyangj/model_10210132/inference_55000,1,0.788230,0.190931,0.072818,0.861572,1.649802,0.263749,0.995917,0.831604,0.848871,0.843631


In [12]:
DiceWeighted = [r['Dice1']+r['Dice2']+r['Dice3']/2 for i,r in eval_df.iterrows()]
eval_df.insert(loc=13,column='DiceWeighted',value=DiceWeighted)
eval_df.sort_values('DiceWeighted',ascending=False).iloc[:,0:14]

ValueError: cannot insert DiceWeighted, already exists

In [13]:
eval_df.sort_values('DiceWeighted',ascending=False).iloc[:,0:14]

,model,inf_dir,test_im,test_11,test_12,test_21,test_22,test_11+22,test_12+21,Dice0,Dice1,Dice2,Dice3,DiceWeighted
12,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_10112055,1,0.861016,0.125160,0.120896,0.821308,1.682324,0.246056,0.995358,0.843021,0.846834,0.845988,2.112849
14,10112055,/proj/NIRAL/users/siyangj/model_10112055/inference_21900_normal,1,0.861763,0.124861,0.121004,0.822459,1.684223,0.245865,0.991524,0.843291,0.846411,0.833207,2.106306
70,10210132,/proj/NIRAL/users/siyangj/model_10210132/inference_55000,1,0.788230,0.190931,0.072818,0.861572,1.649802,0.263749,0.995917,0.831604,0.848871,0.843631,2.102291
63,10210132,/proj/NIRAL/users/siyangj/model_10210132/inference_45000,1,0.798356,0.177805,0.075448,0.843776,1.642132,0.253253,0.994820,0.835899,0.845961,0.837082,2.100401
61,10210130,/proj/NIRAL/users/siyangj/model_10210130/inference_55000,1,0.788889,0.194381,0.081139,0.883951,1.672840,0.275520,0.994809,0.827094,0.851354,0.843518,2.100207
57,10210052,/proj/NIRAL/users/siyangj/model_10210052/inference_70000,1,0.795792,0.186408,0.098338,0.843807,1.639600,0.284747,0.996155,0.818509,0.840425,0.852010,2.084938
54,10210050,/proj/NIRAL/users/siyangj/model_10210050/inference_47700,1,0.787571,0.184371,0.088806,0.862547,1.650118,0.273177,0.992172,0.821132,0.846978,0.825889,2.081055
68,10210054,/proj/NIRAL/users/siyangj/model_10210054/inference_57300,1,0.794176,0.185833,0.093085,0.852272,1.646448,0.278917,0.995408,0.821614,0.839846,0.833980,2.078450
23,10192004,/proj/NIRAL/users/siyangj/model_10192004/inference_6600,1,0.769910,0.198151,0.084447,0.859092,1.629002,0.282597,0.995134,0.813365,0.843567,0.837692,2.075777
58,10210054,/proj/NIRAL/users/siyangj/model_10210054/inference_50000,1,0.829719,0.155244,0.121035,0.832751,1.662470,0.276280,0.995374,0.823699,0.834863,0.831108,2.074117
